In [ ]:
# 1) dates should be formatted in excel to yyyy-MM-dd
# 2) data input as excel format

In [ ]:
%reset -f

In [1]:
import pandas as pd
import numpy as np
import os

from datetime import datetime

In [2]:
filename = "//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/CMP/Input/Current_Yok_Adv.csv"
x1 = pd.read_csv(filename)
Yoko_All_Data = pd.DataFrame(x1)
Yoko_Q1_Data = Yoko_All_Data[Yoko_All_Data['CalendarYearQuarter'].str.contains('Q1')].reset_index().drop('index', axis =1)
Yoko_Q2_Data = Yoko_All_Data[Yoko_All_Data['CalendarYearQuarter'].str.contains('Q2')].reset_index().drop('index', axis =1)
Yoko_Q3_Data = Yoko_All_Data[Yoko_All_Data['CalendarYearQuarter'].str.contains('Q3')].reset_index().drop('index', axis =1)
Yoko_Q4_Data = Yoko_All_Data[Yoko_All_Data['CalendarYearQuarter'].str.contains('Q4')].reset_index().drop('index', axis =1)

In [3]:
custKey = "//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/CMP/Updated Combined CMP Key.xlsx"
x2 = pd.read_excel(custKey)

In [4]:
All_Cust_List = pd.DataFrame(x2)

In [5]:
Yoko_Cust_List = All_Cust_List[All_Cust_List['Program Company'].str.contains('Yokohama')].drop(
    ['CustID','CustAddID','CustID-CustAddID','Program Company','Program Dealer Name','Programs Title','Active Sales?',
     'Report Filter', 'Goodyear TSN'], axis=1).reset_index([]).rename(index=str, columns={
        'CustAddKey':'CustomerAddressKey'})

In [6]:
tt = datetime.now().timetuple()

In [7]:
currentYokoData =pd.DataFrame([])
if (tt.tm_mon == 1) | (tt.tm_mon == 2) | (tt.tm_mon == 3):
    if ((tt.tm_year % 400 == 0) | (tt.tm_year % 4 == 0)) & (tt.tm_year % 100 != 0):
        quarter = 1
        daysINqtr = 91
        currentYokoData = Yoko_Q1_Data
        print('Leap year')
    else:
        quarter = 1
        daysINqtr = 90       
        currentYokoData = Yoko_Q1_Data
elif (tt.tm_mon == 4) | (tt.tm_mon == 5) | (tt.tm_mon == 6):
    quarter = 2
    daysINqtr = 91
    subtractDays = 90          
    currentYokoData = Yoko_Q2_Data
elif (tt.tm_mon == 7) | (tt.tm_mon == 8) | (tt.tm_mon == 9):
    quarter = 3
    daysINqtr = 92
    subtractDays = 90 + 91
    currentYokoData = Yoko_Q3_Data
else:
    quarter = 4
    daysINqtr = 92
    subtractDays = 90 + 91 + 92
    currentYokoData = Yoko_Q4_Data

In [8]:
filename2 = '//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/CMP/Incentive Requirments/YOKO_ADV_Incentive.txt'
df_tierMap = pd.read_csv(filename2, sep = '\t')

In [9]:
tup_now = datetime.now().timetuple()
currentYear = datetime.now().year

In [10]:
ts_currentData = os.path.getmtime(filename)
dt_currentData = datetime.fromtimestamp(ts_currentData)
print(dt_currentData)
ts_incentive = os.path.getctime(filename2)
dt_incentive = datetime.fromtimestamp(ts_incentive)
# print(dt_incentive)

2019-07-11 14:05:16.996709


In [11]:
currentYokoData.head()

,CustomerAddressKey,CalendarYearQuarter,Total_Rev,Total_Net_Qty,Tier,Rebate
0,19,2019-Q3,1016.46,10.0,0,0.0
1,156,2019-Q3,271.00,2.0,0,0.0
2,310,2019-Q3,464.84,4.0,0,0.0
3,377,2019-Q3,62.88,1.0,0,0.0
4,474,2019-Q3,-622.44,-4.0,0,0.0


In [12]:
calculatedData = pd.DataFrame([])
for ii in range(currentYokoData.shape[0]):
# for ii in range(2,3):    

    singleCusty = currentYokoData.iloc[[ii]].assign(QTDsum=0, nextTier=0)

    singleCusty.reset_index(inplace=True,drop=True)

    singleCusty.QTDsum = currentYokoData.iloc[ii,3]
    
#     singleCusty.QTDsum = currentYokoData.Total_Net_Qty
    singleCusty_select = singleCusty[
        ['CustomerAddressKey','CalendarYearQuarter','Total_Rev','Total_Net_Qty','Tier','Rebate','nextTier']]
    singleCusty_select = singleCusty_select.rename(index=str, columns={
        'CustomerAddressKey':'CustomerAddressKey','CalendarYearQuarter':'QTR','Total_Rev':'Total_Sales',
        'Total_Net_Qty':'Total_Qty','Tier':'Tier', 'Rebate':'Rebate', 'nextTier':'nextTier'})

    if  tt.tm_year == currentYear:
        
#         if (tt.tm_mon == 1) | (tt.tm_mon == 2) | (tt.tm_mon == 3):
#             if ((tt.tm_year % 400 == 0) | (tt.tm_year % 4 == 0)) & (tt.tm_year % 100 != 0):
#                 quarter = 1
#                 daysINqtr = 91
                
#                 print('Leap year')
#             else:
#                 quarter = 1
#                 daysINqtr = 90           
#         elif (tt.tm_mon == 4) | (tt.tm_mon == 5) | (tt.tm_mon == 6):
#             quarter = 2
#             daysINqtr = 91
#             subtractDays = 90          
#         elif (tt.tm_mon == 7) | (tt.tm_mon == 8) | (tt.tm_mon == 9):
#             quarter = 3
#             daysINqtr = 92
#             subtractDays = 90 + 91
#         else:
#             quarter = 4
#             daysINqtr = 92
#             subtractDays = 90 + 91 + 92
        
        numDays = tt.tm_yday - subtractDays # get number of days
        estimatedTires = np.floor((singleCusty.QTDsum/(numDays/daysINqtr))).values
        estimatedSales = (singleCusty.Total_Rev/(numDays/daysINqtr)).values
        tier = singleCusty.Tier
        nextTier = singleCusty.Tier + 1
        unitsToNextTier = (df_tierMap.iloc[nextTier,2]) - singleCusty.Total_Net_Qty.values

        if   (estimatedTires >= (df_tierMap.iloc[1,2] )) & (estimatedTires <= (df_tierMap.iloc[1,3])):
            tierType = 1
        elif (estimatedTires >= (df_tierMap.iloc[2,2] )) & (estimatedTires <= (df_tierMap.iloc[2,3])):
            tierType = 2
        elif (estimatedTires >= (df_tierMap.iloc[3,2] )) & (estimatedTires <= (df_tierMap.iloc[3,3])):
            tierType = 3
        elif (estimatedTires >= (df_tierMap.iloc[4,2])):
            tierType = 4
        else:
            tierType = 0
        
        if tierType < 4:
            tierUP = tierType + 1
        else:
            tierUP = 4
            
         # floor to underestimate estimated tires
        total_est_tires = np.floor((singleCusty.QTDsum/(numDays/daysINqtr)).values)

        total_est_tires_rebate = (estimatedSales * df_tierMap.iloc[tierType,1])
        total_per_tire_est_sales = (estimatedSales/total_est_tires)
        # ceil to make sure the customer surpasses the minimal 1up tire threshold
        total_1up_tires = np.ceil((df_tierMap.iloc[tierUP,2])) - total_est_tires
        total_1up_tires_rebate = np.ceil((df_tierMap.iloc[tierUP,2])) * total_per_tire_est_sales * df_tierMap.iloc[tierUP,1]    
        
        if estimatedTires > np.ceil((df_tierMap.iloc[tierUP,2])):
            total_1up_tires = estimatedTires
            total_1up_tires_rebate = total_est_tires_rebate

        tempData = pd.DataFrame(
            {'estimated_tier':tierType, 'current_est_tires':total_est_tires, 'total_est_rebate': total_est_tires_rebate, 
             'Up_tier':tierUP, 'total_up_tires':total_1up_tires, 'total_up_rebate':total_1up_tires_rebate, 
             'period': quarter, 'grain': 'Quarter','unitsToNextTier':unitsToNextTier})
        calculatedData = calculatedData.append(tempData)
        
    else: 
        tempData = pd.DataFrame(
            {'estimated_tier':0, 'current_est_tires':0, 'total_est_rebate': 0, 
             'Up_tier':1, 'total_up_tires':df_tierMap.iloc[1,2], 'total_up_rebate':0, 
             'period': quarter, 'grain': 'Quarter','unitsToNextTier':0})
        calculatedData = calculatedData.append(tempData)   
###########################################################################################################################        
calculatedData = calculatedData.reset_index(drop=True)
compliedData = pd.concat([calculatedData,currentYokoData], axis=1)
compliedData['Date'] = dt_currentData

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:69: RuntimeWarning: invalid value encountered in true_divide
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:69: RuntimeWarning: divide by zero encountered in true_divide


In [15]:
currentYokoData.head()

,CustomerAddressKey,CalendarYearQuarter,Total_Rev,Total_Net_Qty,Tier,Rebate
0,19,2019-Q3,1016.46,10.0,0,0.0
1,156,2019-Q3,271.00,2.0,0,0.0
2,310,2019-Q3,464.84,4.0,0,0.0
3,377,2019-Q3,62.88,1.0,0,0.0
4,474,2019-Q3,-622.44,-4.0,0,0.0


In [13]:
finalData = pd.merge(compliedData, Yoko_Cust_List, on='CustomerAddressKey', how='inner')
finalData = finalData.replace([np.inf, -np.inf], np.nan).fillna(0)
finalData.head()

,estimated_tier,current_est_tires,total_est_rebate,Up_tier,total_up_tires,total_up_rebate,period,grain,unitsToNextTier,CustomerAddressKey,CalendarYearQuarter,Total_Rev,Total_Net_Qty,Tier,Rebate,Date,Program CustID
0,0,57.0,0.000000,1,43.0,307.612895,3,Quarter,90.0,19,2019-Q3,1016.46,10.0,0,0.0,2019-07-11 14:05:16.996709,14435
1,0,34.0,0.000000,1,66.0,287.801029,3,Quarter,94.0,2286,2019-Q3,567.26,6.0,0,0.0,2019-07-11 14:05:16.996709,14344
2,0,57.0,0.000000,1,43.0,274.208421,3,Quarter,90.0,2650,2019-Q3,906.08,10.0,0,0.0,2019-07-11 14:05:16.996709,14257
3,0,5.0,0.000000,1,95.0,469.510500,3,Quarter,99.0,2672,2019-Q3,136.09,1.0,0,0.0,2019-07-11 14:05:16.996709,14008
4,1,126.0,431.934825,2,124.0,1142.684722,3,Quarter,78.0,2801,2019-Q3,2503.97,22.0,0,0.0,2019-07-11 14:05:16.996709,13324


In [ ]:
path = '//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/CMP/python_csv_output/'
writeFilename = path + 'CMP_YOKO_ADV.csv'
finalData.to_csv(writeFilename, index=False)
writeFilename_archive = path + 'csv_archive/' + str(tup_now.tm_year) + '_' + str(tup_now.tm_mon).zfill(2) \
                        + '_' + str(tup_now.tm_mday).zfill(2) + '_' + 'CMP_YOKO_ADV_' \
                        + str(tup_now.tm_hour).zfill(2) + str(tup_now.tm_min).zfill(2) + str(tup_now.tm_sec).zfill(2) \
                        + '.csv'
finalData.to_csv(writeFilename_archive, index=False)

# path = '//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/CMP/python_csv_output/csv_quarter_history/'
# writeFilename = path + 'CMP_YOKO_ADV_historic.csv'
# df_historic.to_csv(writeFilename, index=False)
# writeFilename_archive = path + 'csv_historic_archive/' + str(tup_now.tm_year) + '_' + str(tup_now.tm_mon).zfill(2) \
#                         + '_' + str(tup_now.tm_mday).zfill(2) + '_' + 'CMP_YOKO_ADV_historic_' \
#                         + str(tup_now.tm_hour).zfill(2) + str(tup_now.tm_min).zfill(2) + str(tup_now.tm_sec).zfill(2) \
#                         + '.csv'
# df_historic.to_csv(writeFilename_archive, index=False)

In [ ]:
# for sheet in sheetList:
#     if '1st' in sheet and 'Qtr' in sheet:
#         print('Q1 exist')
#         df1 = pd.read_excel(filename,sheet_name=sheet,header=1)
        
#         df1.columns = df1.columns.str.replace(' ', '_')
#         df1.columns = df1.columns.str.replace('#', 'num')
        
#         df_current = df1
        
#     elif '2nd' in sheet and 'Qtr' in sheet:
#         print('Q2 exist')
#         df2 = pd.read_excel(filename,sheet_name=sheet,header=1)
#         df2.columns = df2.columns.str.replace(' ', '_')
#         df2.columns = df2.columns.str.replace('#', 'num')
        
#         df_historic1 = df1.iloc[:,[1,14,15,16,17,18,19,20]]
#         df_historic1 = df_historic1.add_suffix('_Q1')
        
#         df_current = df2
        
#     elif '3rd' in sheet and 'Qtr' in sheet:
#         print('Q3 exist')
#         df3 = pd.read_excel(filename,sheet_name=sheet,header=1)
#         df3.columns = df3.columns.str.replace(' ', '_')
#         df3.columns = df3.columns.str.replace('#', 'num')
        
#         df_historic2 = df2.iloc[:,[1,14,15,16,17,18,19,20]]
#         df_historic2 = df_historic2.add_suffix('_Q2')
        
#         df_current = df3
        
#     elif '4th' in sheet and 'Qtr' in sheet:
#         print('Q4 exist')
#         df4 = pd.read_excel(filename,sheet_name=sheet,header=1)
#         df4.columns = df4.columns.str.replace(' ', '_')
#         df4.columns = df4.columns.str.replace('#', 'num')
            
#         df_historic3 = df3.iloc[:,[1,14,15,16,17,18,19,20]]
#         df_historic3 = df_historic3.add_suffix('_Q3')
        
#         df_current = df4

In [ ]:
# if 'df_historic1' in locals() and 'df_historic2' in locals() and 'df_historic3' in locals():
#     df_historic = df_historic1.set_index('G3X_Common_Owner_Q1').join(
#         df_historic2.set_index('G3X_Common_Owner_Q2'),how='outer').join(
#         df_historic3.set_index('G3X_Common_Owner_Q3'),how='outer')
    
# elif 'df_historic1' in locals() and 'df_historic2' in locals():
#     print('2 historical quarters')
#     df_historic = df_historic1.set_index('G3X_Common_Owner_Q1').join(
#         df_historic2.set_index('G3X_Common_Owner_Q2'),how='outer').assign(Incentive_Curr_Tier_Q3=0,
#         Net_Shipments_QTD_Q3=0,Units_to_Incnt_Nxt_Tier_Q3=0,Goodyear_Core_Net_Purchases_QTD_Q3=0,
#         Kelly_Net_Purchases_QTD_Q3=0,Other_Net_Purchases_QTD_Q3=0,Incentive_Payment_Q3=0)

# else:
#     print('1 historical quarter')
#     df_historic = df_historic1.assign(
#         Incentive_Curr_Tier_Q2=0,Net_Shipments_QTD_Q2=0,Units_to_Incnt_Nxt_Tier_Q2=0,Goodyear_Core_Net_Purchases_QTD_Q2=0,
#         Kelly_Net_Purchases_QTD_Q2=0,Other_Net_Purchases_QTD_Q2=0,Incentive_Payment_Q2=0,Incentive_Curr_Tier_Q3=0,
#         Net_Shipments_QTD_Q3=0,Units_to_Incnt_Nxt_Tier_Q3=0,Goodyear_Core_Net_Purchases_QTD_Q3=0,
#         Kelly_Net_Purchases_QTD_Q3=0,Other_Net_Purchases_QTD_Q3=0,Incentive_Payment_Q3=0)

In [ ]:
# for ii in range(Yoko_Q1_Data.shape[0]):
#         if Yoko_Q1_Data.Tier[ii] == 0:
#             Up_Tier = 1
#         elif Yoko_Q1_Data.Tier[ii] == 1:
#             Up_Tier = 2